# Face Recognition

In [1]:
import cv2  #import OpenCV module
import os   #import os module for reading training data directories and paths
import numpy as np

### Training Data

In [2]:
subjects = [1]
subjects_names = ["Rachel"]

### Detecting faces

In [3]:
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2_base_dir = os.path.dirname(os.path.abspath(cv2.__file__))
    lbp_model = os.path.join('lbpcascade_frontalface.xml')
    face_cascade = cv2.CascadeClassifier(lbp_model)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    if (len(faces) == 0):
        return None, None
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

This function reads training images, returns one list with faces and labels.

In [4]:
def prepare_training_data(data_folder_path):
    
    # get the directories (one directory for each subject) in data folder
    dirs = ['Rachel'] # os.listdir(data_folder_path)
    print(dirs)
    faces = []   #list to hold all subject faces  
    labels = []  #list to hold labels for all subjects
    # our subject directories start with the letter 's' so we should ignore any non-relevant directories if any
    label = 0
    subject_dir_path = data_folder_path + "/" + dirs[0]

    # find image names in the directory
    subject_images_names = os.listdir(subject_dir_path)
    print(subject_images_names)    
    for image_name in subject_images_names:
        if image_name.startswith("."):   #ignore system files like .DS_Store
            continue;
        image_path = subject_dir_path + "/" + image_name
        # read the image
        image = cv2.imread(image_path)

        # show the image
        cv2.imshow("Training on image...", image)
        cv2.waitKey(100)


        # detect each face in the images
        face, rect = detect_face(image)

        if face is not None: # ignore images without face
            # add face to list of faces
            faces.append(face)
            # add label for this face
            labels.append(label)

    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

In [5]:
faces, labels = prepare_training_data("data/dataset")

['Rachel', 'Ross']
Rachel
['rachel2.jpeg', 'rachel1.jpg', 'rachel3.jpg', 'rachel6.jpg', 'rachel5.png', 'rachel4.jpg']
Ross
['ross.png', 'ross6.jpeg', 'ross4.jpeg', 'ross5.jpg', 'ross2.jpg', 'ross3.jpeg', 'ross1.jpg']


In [ ]:
print("Preparing data...")
faces, labels = prepare_training_data("data/dataset")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))


In [7]:
#create our LBPH face recognizer 
face_recognizer = cv2.face.LBPHFaceRecognizer_create() 

#or use EigenFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face.createEigenFaceRecognizer()

#or use FisherFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face.createFisherFaceRecognizer()

In [8]:
# train our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))

### Prediction

In [9]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
# function to draw text on give image starting from (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [10]:
def predict(test_img):
    img = test_img.copy()
    face, rect = detect_face(img)

    label, conf= face_recognizer.predict(face)
    label_text = subjects_names[label]
    
    draw_rectangle(img, rect)
    draw_text(img, label_text, rect[0], rect[1]-5)
    
    return img, label_text

In [ ]:
print("Predicting images...")

# load test images
test_img1 = cv2.imread("data/rachel.jpg")

# recognize the person
predicted_img1, label_text = predict(test_img1)
print("Prediction complete")

cv2.imshow(label_text, predicted_img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

Predicting images...
[1, 2]
0
Prediction complete
